(gpio-target)=
# GPIO

## ESPDuino32 pinnen

De ESP32 heeft redelijk wat GPIO (General Purpose Input Output) pinnen. Sommige pinnen ondersteunen enkel één richting, sommige zijn strapping pins, nog andere hebben alternatieve functies, ...  Op de website van [RandomNerdTurtorials](https://randomnerdtutorials.com/esp32-pinout-reference-gpios/) staat hierover veel informatie, en een heel handig overzicht in tabelvorm. De belangrijkste zaken zijn hieronder overgenomen:

```{figure} ./images/esp32_pinout.png
:width: 500px
:align: left
:figwidth: image
:figclass: myBlockImg

ESP32 en mogelijkheden per GPIO.
```

### Input only

De pinnen 34, 35, 36 en 39 zijn enkel ingangen. Deze kunnen niet gebruikt worden als uitgang. Er zal echter geen fout optreden indien we dit proberen. Er zal simpelweg niets veranderen aan de logische toestand van de pin. Deze pinnen bevatten eveneens geen pull-up of pull-down weerstanden. Indien je dit nodig hebt zul je deze zelf extern moeten toevoegen.

### Capacitive touch

De pinnen 0, 2, 4, 12, 13, 14, 15, 27, 22 en 33 (10 in totaal) hebben de mogelijkheid om elektrostatische lading te detecteren. Indien een lange kabel, een stukje metaal of gelijk welk geleidend materiaal wordt aangebracht aan deze pin, kan het aanraken met de hand van deze de lading veranderen. Dit kan vervolgens gedetecteerd worden in de ESP. Opgelet, veel van deze pinnen zijn _strapping pins_. Zie hiervoor verder.

### Analog to Digital (ADC)

Er zijn in totaal 18 pinnen voor ADC beschikbaar, en dit verdeeld over twee banken (ADC1 en ADC2). In principe zitten er dus slecht twee ADC convertoren in de ESP, die vooraf worden gegaan door een multiplexer. De ADC1 heeft een multiplexer met 8 beschikbare ingangen, de ADC2 een multiplexer met 10 beschikbare ingangen. De voorkeur moet gegeven worden aan ADC1, aangezien enkel deze beschikbaar is wanneer WiFi wordt gebruikt. Voor IoT toepassingen zal WiFi altijd nodig zijn, dus is de keuze voor de ADC1 aangewezen. Indien men toch meer analoge ingangen nodig heeft zal de WiFi (tijdelijk) moeten gedeactiveerd worden.

* ADC1: 32, 33, 34, 35, 36, 37, 38, 39
* ADC2: 0, 2, 4, 12, 13, 14, 15, 25, 26, 27 (veel van deze stemmen overeen met de capacitive touch)

### Digital to Analog (DAC)

Voor de omgekeerde weg zijn er slechts twee mogelijkheden (zonder demultiplexer), namelijk DAC1 (pin 25) en DAC2  (pin 26). Herrouteren van deze signalen is moeilijk, dus deze pinnen moeten hiervoor altijd gebruikt worden. Deze zitten overigens enkel en alleen maar op de ESP32, en niet op de light varianten (ESP32-S en ESP32-C). Indien je voor een project gebruik maakt van een alternatieve ESP32 wordt hier best rekening mee gehouden.

### PWM

De ESP32 heeft intern 16 PWM controllers, waarbij iedere uitgangspin kan gebruikt worden om één van de 16 controller uitgangen mee te verbinden.

### Strapping pins

Iets die speciaal/eigenaardig is aan de ESP zijn de _strapping pins_. Dit zijn pinnen die tijdens het booten op logisch niveau worden ingelezen. Door een specifiek logisch niveau aan te leggen op deze pinnen wordt de ESP in een bepaalde _boot mode_ gebracht. _De belangrijkste boot mode voor ons is 0x13 (SPI_FAST_FLASH_BOOT), die het programma laadt vanuit flash geheugen. Het "programma" die voor ons interessant is, is MicroPython._

```{figure} ./images/bootmode.png
:width: 500px
:align: left
:figwidth: image
:figclass: myBlockImg

Indicatie van bootmode op de CLI tijdens booten van de ESP32.
```

Mocht MicroPython niet starten zoals verwacht, dan kan in de CLI gecontroleerd worden wat fout loopt. Meestal zal de _boot mode_ afwijken van de verwachtte 0x13, en dit wellicht te wijten aan een elektrische verbindingen aangebracht op de _strapping pins_.

Volgende pinnen met hun overeenkomstige boot waarde bestaan:
* 0x01: GPIO5
* 0x02: GPIO15
* 0x04: GPIO4
* 0x08: GPIO2
* 0x10: GPIO0
* 0x20: GPIO12

Om 0x13 te bekomen hebben we nood aan 0x10 + 0x02 + 0x01. Dit betekent dat GPIO0, GPIO15 en GPIO5 tijdens het booten logisch 1 moeten zijn (gestrapped naar de voeding), en alle andere pinnen logisch 0 (gestrapped naar de massa). Mocht er per ongeluk een van de pinnen foutief hoog of laag gemaakt zijn tijdens het booten, dan zal een andere boot mode getoond worden. Via bovenstaand overzicht zou het probleem snel achterhaald moeten kunnen worden.

## GPIO klasse

(micro)Python werkt op de achtergrond altijd met klasses. We konden dit reeds eerder zien via de functie `type()` die ons het type van variabelen liet achterhalen. Volgend voorbeeld geeft ons het type van een string terug.

In [1]:
myStr = "Koen"
print(type(myStr))

<class 'str'>


Hier is duidelijk te zien dat het type van een string eigenlijk een specifieke klasse is. Van iedere klasse kunnen er objecten gemaakt worden. `myStr` is dus een object van de klasse `str`, en bij deze klasse komen automatisch een aantal methoden die op het object kunnen gebruikt worden.

### Import

Om gebruik te kunnen maken van de GPIO functionaliteiten in MicroPython moeten we allereerst de klasse `Pin` importeren vanuit de `machine` klasse. Deze klasse omvat alle functionaliteiten die een standaard GPIO aanbiedt (dus basis in- en uitgang). Deze klasse is identiek voor zowel een in- als een uitgang op de ESP.

```python
from machine import Pin
```

Deze manier geeft ons meteen toegang tot de `Pin`-klasse. Indien je ook nog andere zaken van de `machine` bibliotheek wil gebruiken kun je ook opteren om gewoon deze volledig te importeren.

```python
import machine
```

:::{admonition} Opgelet
:class: warning
Door volledig `machine` te importeren moet je nu wel gebruik maken van de correcte/volledige *namespace*. Deze wordt nu `machine.Pin`, en niet meer `Pin` alleen. 
:::

### Object

Het aanmaken van een object van de klasse `Pin` bestaat door het aanroepen van de constructor. De constructor verwacht echter een aantal argumenten. 

`class machine.Pin(id, mode=-1, pull=-1, *, value=None, drive=0, alt=-1)`

De constructor heeft één verplicht argument (die geen *default value* heeft), namelijk het argument `id`. De overige argumenten worden ingesteld op de waarde die er bij staat, maar kan door de gebruiker *overschreven* worden door deze ook op te geven.

* `id` is verplicht en kan een willekeurige integer zijn die het juiste pinnummer op de ESP32 aangeeft. Het gedeelte `IO` die voor het pinnummer staat mag niet mee opgegeven worden.
* `mode` stelt de juiste mode in van hoe de pin zich moet gedragen. Volgende modes zijn beschikbaar:
    * `Pin.IN`: Wordt gebruikt als je de pin wil gebruiken als een ingang (om bijvoorbeeld een drukknop of sensor in te lezen). De impedantie van de pin wordt hierdoor hoog.
    * `Pin.OUT`: wordt gebruikt als je de pin wil gebruiken als een uitgang (om bijvoorbeeld een LED of een motor aan te sturen).
    * `Pin.OPEN_DRAIN`: Wordt gebruikt als je de pin wil gebruiken als een uitgang die geen stroom kan sourcen (enkel maar sinken). Dit stemt overeen met een *open collector* uitgang die wordt behandeld op het einde van het 2de leerjaar van de 3de graad. Typische toepassingen is bij buscommunicatie of het aansturen van actuatoren op een hogere spanning dan deze van de mircocontroller zelf.
* `pull` laat toe een interne zwakke *pull-up* of interne *pull-down* weerstand te verbinden met de pin, zodat je als gebruik extern zelf geen hoeft aan te sluiten. Zie hiervoor verder bij het onderdeel [outputs](project:#output-target).
    * `None` laat de pin zwevend (hoog impedant).
    * `Pin.PULL_UP` activeert een hoog-ohmige *pull-up* weerstand.
    * `Pin.PULL_DOWN` activeert een hoog-ohmige *pull-down* weerstand.
* `value` is enkel geldig wanneer je te maken hebt met een `Pin.OUT` of `Pin.OPEN_DRAIN`, en plaatst de pin op een specifieke waarde (logisch 0 of logisch 1). Indien niets opgegeven wordt zal de waarde onveranderd blijven. Bij het aanmaken van het object zal dit op een specifieke waarde worden geplaatst (wellicht logisch 0), maar dit kan handig zijn bij herinitialisatie.
* `drive` laat toe de uitgangsimpedantie te wijzigen. Je kan dit zien als een Thévenin bron die 3,3V levert, maar waarbij de Thévenin weerstand instelbaar is op enkele vaste waarden. Standaard staat dit op `DRIVE_0`. Er is geen reden dit anders in te stellen!
    * `Pin.DRIVE_0` $I_k=11mA$, wat ongeveer overeen komt met een $R_{th}=300\mathsf{\Omega}$
    * `Pin.DRIVE_1` $I_k=23mA$, wat ongeveer overeen komt met een $R_{th}=150\mathsf{\Omega}$
    * `Pin.DRIVE_2` $I_k=46mA$, wat ongeveer overeen komt met een $R_{th}=75\mathsf{\Omega}$
    * `Pin.DRIVE_3` $I_k=92mA$, wat ongeveer overeen komt met een $R_{th}=37\mathsf{\Omega}$

### Methoden

De methoden hangen af van of je de pin gebruikt als een in- of een output. Volgende methoden zijn beschikbaar (NOG TOE TE LICHTEN):

* `Pin.init(...)`
* __`Pin.value(x)`__
* __`Pin.on()`__
* __`Pin.off()`__
* `Pin.irq(...)`
* `Pin.mode(x)`
* `Pin.pull(x)`
* `Pin.drive(x)`
* __`Pin.toggle()`__

(output-target)=
## Outputs

### Digitaal (blink)

Als eerste toepassing (die jullie nog te goed hadden) volgt hierbij de code die nodig is om een digitale uitgang aan te sturen, en die we meteen gaan uitbreiden naar een eerste toepassing, namelijk het laten knipperen van een LED.

:::{admonition} Tip
:class: hint
Op het ESPduino bordje zit een LED die kan gebruikt worden als debug LED. Deze bevindt zich op GPIO2.
:::


De code bestaat uit enkele stappen:
1) Inladen GPIO functionaliteit
2) Inladen tijdsfunctionaliteit (zie hiervoor [tijd](project:#tijd-target))
3) Aanmaken Pin object
4) Pin object togglen in een loop

Gebruiken we eerder geziene skeleton:
```python
#imports
from machine import Pin
from time import sleep_ms as delay

#constants & objects
LED = 2
myDbgLed = Pin(LED,mode=Pin.OUT)

#main
def main():
    while(True):
        myDbgLed.toggle()
        delay(250)

#oproepen main
if(__name__=="__main__"):
    main()
```

### PWM

TODO

### Analoog (DAC)

TODO

## Inputs

### Digitaal

Een toepasselijk voorbeeld voor een digitale input is het binnenlezen van de status van een drukknop. Deze drukknop kan (elektrisch) op twee verschillende manieren aangesloten zijn op de hardware:

```{figure} ./images/digital_input.png
:width: 500px
:align: left
:figwidth: image
:figclass: myBlockImg

Mogelijkheden tot aansluiten van een drukknop.
```
Bovenstaande methoden zullen beiden ongeveer identiek werken, m.u.v. het ingelezen logische niveau tijdens bediening. De rechtste implementatie zal een logische 1 genereren wanneer ingedrukt, terwijl de linkse implementatie een logische 0 zal generen. De (optionele) weerstand $R_1$ is hierbij een _pullup_ of _pulldown_ weerstand. De (optionele) weerstand $R_2$ een stroombeveiliging, en dit wanneer de eindgebruiker foutief een ingang als uitgang zou definiëren. _In beide gevallen wordt gemeld dat de weerstand optioneel is. Men kan opteren om beide weg te laten indien de code correct wordt geschreven. In uiteindelijke implementaties zorgen minder componenten voor grotere [MTBF](https://nl.wikipedia.org/wiki/Mean_time_between_failures) cijfers, wat natuurlijk altijd een meerwaarde is voor het product._

Als eerst moeten we een object aanmaken van de klasse `Pin` die onze digitale ingang voorstelt. Dit kan als volgt:

```python
DRUKKNOP = 26
myPushButton = Pin(DRUKKNOP, mode=Pin.IN)
```

Voor bijna alle pinnen, m.u.v. de pinnen 34, 35, 36 & 39, kan er geopteerd worden om een pullup of pulldown weerstand intern in de ESP te activeren, zodat de externe weerstand kan weggelaten worden. Dit kan door de constructor van de `Pin` uit te breiden als volgt:

```python
myPushButton = Pin(DRUKKNOP, mode=Pin.IN, pull=Pin.PULL_UP) #Pin.PULL_DOWN bestaat eveneens
```

Het binnenlezen van de logische waarde kan vervolgens via de methode `value()`:

```python
print(myPushButton.value())
```

Indien we deze code nu opnemen in onze skeleton, en nog uitbreiden met een herhaling krijgen we volgende implementatie:

```python
#imports
from machine import Pin

#constants & objects
DRUKKNOP = 26
myPushButton = Pin(DRUKKNOP, mode=Pin.IN, pull=Pin.PULL_UP)

#main
def main():
    lastValue = myPushButton.value()
    while(True):
        if(lastValue != myPushButton.value()):
            lastValue = myPushButton.value()
            print(lastValue)

#oproepen main
if(__name__=="__main__"):
    main()
```

### Interrupts

De voortdurende controle of een ingang al dan niet van logische waarde verandert vraagt redelijk wat CPU tijd. Deze methode wordt in het vakjargon _polling_ genaamd, waarbij de status van de ingangspin in de `main()` voortdurend wordt afgevraagd (_gepolled_). Dit kan voorkomen worden door aan de ingangspin een callback (IRQ, Interrupt ReQuest) te koppelen die wordt opgeroepen wanneer de toestand van de `Pin` wijzigt. 

```python
GPIO26.irq(<callback functie>)
```

Volgende implementatie toont de waarde van de pin wanneer deze wijzigt van toestand:

```python
from machine import Pin

GPIO26 = Pin(26, Pin.IN, Pin.PULL_UP)

def button_isr(context):
    print(context.value())

def main():
    GPIO26.irq(button_isr)
    while(True):
        pass

if(__name__=="__main__"):
    main()
```

Bovenstaande implementatie kan nog uitgebreid worden. Volgende zaken zijn nog van belang bij de `irq` methode:
* `trigger` laat toe om de `callback` slechts op te roepen bij een overgang van/naar een bepaald logisch niveau. via `Pin.IRQ_FALLING` en/of `Pin.IRQ_RISING` kan er geselecteerd worden op een _rising_ of _falling_ edge.
* `priority` laat toe een prioriteit te geven aan de interrupt. Hoe hoger het getal, hoe groter de prioriteit. 

Pas gerust als test bovenstaande code aan als volgt:
```python
GPIO26.irq(button_isr,trigger=Pin.IRQ_FALLING,hard=True)
```

### Analoog (ADC)

TODO